In [1]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install scikit-learn

   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ------- -------------------------------- 1.6/8.7 MB 9.2 MB/s eta 0:00:01
   --------------------- ------------------ 4.7/8.7 MB 13.1 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 14.4 MB/s  0:00:00
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   ---- ----------------------------------- 4.5/38.5 MB 21.2 MB/s eta 0:00:02
   ------- -------------------------------- 7.6/38.5 MB 17.6 MB/s eta 0:00:02
   ------------ --------------------------- 11.8/38.5 MB 18.0 MB/s eta 0:00:02
   ---------------- ----------------------- 15.5/38.5 MB 17.7 MB/s eta 0:00:02
   -------------------- ------------------- 20.2/38.5 MB 18.5 MB/s eta 0:00:01
   -------------------------- ------------- 26.0/38.5 MB 19.9 MB/s eta 0:00:01
   -------------------------------- ------- 30.9/38.5 MB 20.4 MB/s eta 0:00:01
   ------------------------------------- -- 35.7/38.5 MB 20.6 MB/s eta 0:00:01
   -


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from pymongo.errors import OperationFailure
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import os
import io
import pickle

In [ ]:
N = 2000